In [21]:
from autoprotocol import Protocol
p = Protocol()

plate1 = p.ref( 'plate1', cont_type='96-deep', discard=True ) 
plate2 = p.ref( 'plate2', cont_type='96-deep', discard=True ) 
p.transfer( plate1.well('C5'), plate2.well('F3'), '10:microliter' )
p.transfer( plate1.well('F3'), plate1.well('A1'), '19:microliter' )
p.transfer( plate1.well('A1'), plate1.well('G1'), '112:microliter' )

import json
print json.dumps( p.as_dict(), indent=2 )

{
  "refs": {
    "plate2": {
      "new": "96-deep", 
      "discard": true
    }, 
    "plate1": {
      "new": "96-deep", 
      "discard": true
    }
  }, 
  "instructions": [
    {
      "groups": [
        {
          "transfer": [
            {
              "volume": "10.0:microliter", 
              "to": "plate2/62", 
              "from": "plate1/28"
            }
          ]
        }, 
        {
          "transfer": [
            {
              "volume": "19.0:microliter", 
              "to": "plate1/0", 
              "from": "plate1/62"
            }
          ]
        }, 
        {
          "transfer": [
            {
              "volume": "112.0:microliter", 
              "to": "plate1/72", 
              "from": "plate1/0"
            }
          ]
        }
      ], 
      "op": "pipette"
    }
  ]
}


In [22]:
# supported container types 
# just one for now 
cont_types = {
    '96-deep': {
        'plate_height': 34,
        'well_bottom_height': 4, 
        'x_spacing': 9,
        'y_spacing': 9, 
        'num_rows': 8,
        'num_cols': 12,        
    },
}

# define our plate class 
class Plate():
    def __init__( self, cont_type=None, spot=None ):
        # fill these in from container type 
        self.plate_height = cont_types[ cont_type ]['plate_height']
        self.well_bottom_height = cont_types[ cont_type ]['well_bottom_height']
        self.x_spacing = cont_types[ cont_type ]['x_spacing'] 
        self.y_spacing = cont_types[ cont_type ]['y_spacing']
        self.num_rows = cont_types[ cont_type ]['num_rows']
        self.num_cols = cont_types[ cont_type ]['num_cols']

        # from provided spot, calculate offsets
        self.spot = spot
        self.rot = 0 
        self.x, self.y = 0, 0 
        
    def well_id_to_coord( self, well_id ):
        if type( well_id ) is str:
            row = { j: i for i, j in enumerate('ABCDEFGH') }[ well_id[0] ]
            col = int( well_id[1:].strip() ) - 1
        else:
            row = ( well_id / self.num_rows ) 
            col = ( well_id % self.num_cols ) 

        x = col * self.x_spacing 
        y = row * self.y_spacing 
        
        return ( self.x + x, self.y + y )

# split up refs and instructions
refs = p.as_dict()['refs'] 
inst = [ inst for inst in p.as_dict()['instructions'] ]

# make objects
refs_dict = { }
for index, ( ref, dic ) in enumerate( refs.iteritems() ):
    refs_dict.update( { ref: Plate( cont_type=dic['new'], spot=index ) } )

# compile autoprotocol to gcode 
gcode = []
blowout = 3 
transfer_stack = []
for instruction in inst:
    #print instruction
    for group in instruction['groups']:
        #print group
        for transfer in group['transfer']:
            transfer_stack.append( transfer )
            
for transfer in transfer_stack:
    volume, unit = transfer['volume'].split( ':' )
    dest_plate, dest_well = transfer['to'].split( '/' )
    source_plate, source_well = transfer['from'].split( '/' )
    
    print '(begin transfer)'
    print 'G1 Z%0.1f F400 (clear the plate)' % refs_dict[ source_plate ].plate_height
    print 'G1 X%0.1f Y%0.1f F400 (go to source well)' % refs_dict[ source_plate ].well_id_to_coord( int( source_well ) )
    print 'G1 Z%0.1f F400 (dive)' % refs_dict[ source_plate ].well_bottom_height
    print 'G1 A%f F100 (aspirate)' % float( volume )
    print 'G1 Z%0.1f F400 (clear the source plate)' % refs_dict[ source_plate ].plate_height
    print 'G1 X%0.1f Y%0.1f F400 (go to dest well)' % refs_dict[ dest_plate ].well_id_to_coord( int( dest_well ) )
    print 'G1 Z%0.1f F400 (dive into dest)' % refs_dict[ dest_plate ].well_bottom_height
    print 'G1 A%f (dispense)' % float( volume )

(begin transfer)
G1 Z34.0 F400 (clear the plate)
G1 X36.0 Y27.0 F400 (go to source well)
G1 Z4.0 F400 (dive)
G1 A10.000000 F100 (aspirate)
G1 Z34.0 F400 (clear the source plate)
G1 X18.0 Y63.0 F400 (go to dest well)
G1 Z4.0 F400 (dive into dest)
G1 A10.000000 (dispense)
(begin transfer)
G1 Z34.0 F400 (clear the plate)
G1 X18.0 Y63.0 F400 (go to source well)
G1 Z4.0 F400 (dive)
G1 A19.000000 F100 (aspirate)
G1 Z34.0 F400 (clear the source plate)
G1 X0.0 Y0.0 F400 (go to dest well)
G1 Z4.0 F400 (dive into dest)
G1 A19.000000 (dispense)
(begin transfer)
G1 Z34.0 F400 (clear the plate)
G1 X0.0 Y0.0 F400 (go to source well)
G1 Z4.0 F400 (dive)
G1 A112.000000 F100 (aspirate)
G1 Z34.0 F400 (clear the source plate)
G1 X0.0 Y81.0 F400 (go to dest well)
G1 Z4.0 F400 (dive into dest)
G1 A112.000000 (dispense)
